In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2

In [44]:
!unzip drow.zip -d droww

Archive:  drow.zip
 extracting: droww/README.dataset.txt  
 extracting: droww/README.roboflow.txt  
   creating: droww/test/
 extracting: droww/test/1007_jpg.rf.8319a9dffaa9be27f15f895d359caa32.jpg  
 extracting: droww/test/1032_jpg.rf.72e0d99e145f3046bda3badd03b5dafc.jpg  
 extracting: droww/test/1114_jpg.rf.71aa56fa2717215831bf146d4ac58cc7.jpg  
 extracting: droww/test/1130_jpg.rf.a860a3990143e0e9c6e473b55c4e606e.jpg  
 extracting: droww/test/1178_jpg.rf.922b3e2d663575da0796b9813d3dad20.jpg  
 extracting: droww/test/117_jpg.rf.88746f10e82290a4eff4717e4987566f.jpg  
 extracting: droww/test/119_jpg.rf.8c73c3839903dac59546095fb9a78f4a.jpg  
 extracting: droww/test/1247_jpg.rf.b532eccd06e690c2485efab781e1b374.jpg  
 extracting: droww/test/127_jpg.rf.5e0accd8c79501003096fc8e21b1ee39.jpg  
 extracting: droww/test/130_jpg.rf.5ec6041c61d34963d32731918908c8a8.jpg  
 extracting: droww/test/1369_jpg.rf.870dcc7eac2bfc9dcf91e90a3d28af3a.jpg  
 extracting: droww/test/142_jpg.rf.82f2a68f9cf5c29f3f9

In [2]:
train_df = pd.read_csv('drow/train/_annotations.csv')
test_df = pd.read_csv('drow/test/_annotations.csv')
valid_df = pd.read_csv('drow/valid/_annotations.csv')

In [3]:
def load_image(image_path):
    return Image.open(image_path)

In [4]:
from PIL import Image
import os
# Adjust the base path to include 'drow' directory
base_path = 'drow'

# Example to access the first image in the training set
image_path = os.path.join(base_path, 'train', train_df.iloc[0]['filename'])
print(image_path)  # Check what the constructed path looks like
image = load_image(image_path)
image.show()

drow/train/828_jpg.rf.046f0b3f4f82f014d8b7a7a7148469b4.jpg


In [5]:
from PIL import ImageOps

def transform_image(image):
    # Resize image
    image = image.resize((256, 256))
    # Convert to grayscale
    image = ImageOps.grayscale(image)
    return image

# Load an image
image_path = os.path.join(base_path,'train', train_df.iloc[0]['filename'])
image = load_image(image_path)

# Apply transformations
transformed_image = transform_image(image)
transformed_image.show()

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

2024-05-02 23:05:27.883401: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Using sigmoid activation for binary classification
    ])
    return model

In [8]:
from tensorflow.keras.metrics import Precision, Recall
model = build_model((256, 256, 3))  # Adjust the input shape according to your image size
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision()])
model.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,105 (56.61 MB)

 Trainable params: 14,839,105 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1./255)  # No data augmentation for validation/testing

In [10]:
train_df['filename'] = 'drow/train/' + train_df['filename']
valid_df['filename'] = 'drow/valid/' + valid_df['filename']
test_df['filename'] = 'drow/test/' + test_df['filename']

In [11]:
train_df['class'] = train_df['class'].astype(str)
valid_df['class'] = valid_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Prepare the data loaders using the full paths in the DataFrames
train_loader = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    class_mode='binary',
    batch_size=32
)

valid_loader = datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    class_mode='binary',
    batch_size=32
)

test_loader = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    class_mode='binary',
    batch_size=32
)

Found 709 validated image filenames belonging to 2 classes.
Found 203 validated image filenames belonging to 2 classes.
Found 101 validated image filenames belonging to 2 classes.


In [13]:
history = model.fit(
    train_loader,
    epochs=50,
    validation_data=valid_loader
)

Epoch 1/50


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 0.5303 - loss: 1.4871 - precision: 0.4675 - recall: 0.4932 - val_accuracy: 0.5074 - val_loss: 0.6591 - val_precision: 0.5074 - val_recall: 1.0000
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.6150 - loss: 0.6362 - precision: 0.5871 - recall: 0.6492 - val_accuracy: 0.6798 - val_loss: 0.5611 - val_precision: 0.8519 - val_recall: 0.4466
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.7551 - loss: 0.5091 - precision: 0.8091 - recall: 0.6289 - val_accuracy: 0.7094 - val_loss: 0.5323 - val_precision: 0.8548 - val_recall: 0.5146
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.7926 - loss: 0.4369 - precision: 0.8184 - recall: 0.7383 - val_accuracy: 0.7291 - val_loss: 0.4909 - val_precision: 0.8429 - val_recall: 0.5728
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.8505 - loss: 0.3941 - precision: 0.9125 - recall: 0.7738 - val_accuracy: 0.7833 - val_loss: 0.4502 - val_precision:

In [14]:
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_loader)
print('Test accuracy:', test_acc)
print('Test precision:', test_precision)
print('Test recall:', test_recall)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 547ms/step - accuracy: 0.8887 - loss: 0.7810 - precision: 0.8734 - recall: 0.8726
Test accuracy: 0.8910890817642212
Test precision: 0.8799999952316284
Test recall: 0.8979591727256775
